In [ ]:
!nvidia-smi

Wed Mar 12 15:03:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             13W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install ultralytics
!pip install roboflow
!pip install tensorflow
!pip install tensorboard
!pip install numba

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 932.9/932.9 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import shutil
import torch
from ultralytics import YOLO
from roboflow import Roboflow
import random
import shutil
import os
import tensorflow as tf
from numba import cuda

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
shutil.rmtree('/content/runs')

FileNotFoundError: [Errno 2] No such file or directory: '/content/runs'

In [ ]:

rf = Roboflow(api_key="QW8BuuDWpo0lbn99BpZw")
project = rf.workspace("data-science-173-msp9r").project("motorcycle-helmet-detection")
version = project.version(2)
dataset = version.download("yolov11")



loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Motorcycle-Helmet-Detection-2 in yolov11:: 100%|██████████| 4442/4442 [00:00<00:00, 5559.62it/s]


In [ ]:
#split images
src_dir = "/content/Motorcycle-Rider-Helmets-2/train/images"
dest_dir = "/content/Motorcycle-Rider-Helmets-2/val/images"

val_percentage = 10

if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

files = [f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f))]
num_files_to_move = int(len(files) * (val_percentage / 100))

files_to_move = random.sample(files, num_files_to_move)

for file in files_to_move:
    shutil.move(os.path.join(src_dir, file), os.path.join(dest_dir, file))

print(f"Moved {num_files_to_move} files from {src_dir} to {dest_dir}")

Moved 262 files from /content/Motorcycle-Rider-Helmets-2/train/images to /content/Motorcycle-Rider-Helmets-2/val/images


In [ ]:
#split labels

src_dir = "/content/Motorcycle-Rider-Helmets-2/train/labels"
dest_dir = "/content/Motorcycle-Rider-Helmets-2/val/labels"

val_percentage = 10

if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

files = [f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f))]
num_files_to_move = int(len(files) * (val_percentage / 100))

files_to_move = random.sample(files, num_files_to_move)

for file in files_to_move:
    shutil.move(os.path.join(src_dir, file), os.path.join(dest_dir, file))

print(f"Moved {num_files_to_move} files from {src_dir} to {dest_dir}")

Moved 262 files from /content/Motorcycle-Rider-Helmets-2/train/labels to /content/Motorcycle-Rider-Helmets-2/val/labels


In [ ]:
device = cuda.get_current_device()
print(device)
# torch.cuda.empty_cache()
# device.reset()

<CUDA device 0 'b'Tesla T4''>


In [ ]:
os.environ['TORCH_CUDA_ALLOC_SYNC'] = "1"
os.environ['CUDA_LAUNCH_BLOCKING=1'] = "0"
# os.getenv("TORCH_CUDA_ALLOC_SYNC")

ValueError: illegal environment variable name

In [ ]:
print("yahooo")
model = YOLO("yolo11l.pt")

results = model.train(data="/content/Motorcycle-Helmet-Detection-2/data.yaml", epochs=100, imgsz=640)

results = model("/content/test1.png")

100%|██████████| 49.0M/49.0M [00:00<00:00, 354MB/s]


Ultralytics 8.3.88 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11l.pt, data=/content/Motorcycle-Helmet-Detection-2/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sho

100%|██████████| 755k/755k [00:00<00:00, 115MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  2    173824  ultralytics.nn.modules.block.C3k2            [128, 256, 2, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  2    691712  ultralytics.nn.modules.block.C3k2            [256, 512, 2, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  2   2234368  ultralytics.nn.modules.block.C3k2            [512, 512, 2, True]           
  7                  -1  1   2360320  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 243MB/s]


...

     12/100      12.5G      1.734      1.701      1.536         28        640: 100%|██████████| 97/97 [01:11<00:00,  1.36it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:08<00:00,  1.72it/s]

                   all        438       1451      0.558      0.327      0.411      0.226



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     13/100      12.5G       1.72      1.691      1.529         91        640:  94%|█████████▍| 91/97 [01:07<00:04,  1.34it/s]


KeyboardInterrupt: 

In [ ]:
model = YOLO("/content/runs/detect/train/weights/best.pt")  # load a custom trained model

# Export the model
model.export(format="onnx")

Ultralytics 8.3.85 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.00GHz)
YOLO11n summary (fused): 100 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from '/content/runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 8, 8400) (5.2 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 312.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.9/142.9 kB 255.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.8/280.8 MB 288.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 219.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 325.7 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 15.3s, installed 3 packages: ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu']
requirements: ⚠️ Restart runtime or rerun comman

'/content/runs/detect/train/weights/best.onnx'

In [ ]:
model = YOLO("yolo11x.pt")
model = YOLO("best.pt")
results = model.predict("/content/test_image/image2.png",data="data.yaml")

# Access the results
for result in results:
    xywh = result.boxes.xywh  # center-x, center-y, width, height
    xywhn = result.boxes.xywhn  # normalized
    xyxy = result.boxes.xyxy  # top-left-x, top-left-y, bottom-right-x, bottom-right-y
    xyxyn = result.boxes.xyxyn  # normalized
    names = [result.names[cls.item()] for cls in result.boxes.cls.int()]  # class name of each box
    confs = result.boxes.conf  # confidence score of each box

print(xywh)
print(xywhn)
print(xyxy)
print(xyxyn)
print(names)
print(confs)


image 1/1 /content/test_image/image2.png: 640x640 2 persons, 2 cars, 1 motorcycle, 87.1ms
Speed: 2.9ms preprocess, 87.1ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)
tensor([[357.0129, 245.5836,  41.9036,  57.7080],
        [359.6050, 220.8589,  27.6129,  73.7904],
        [374.0677, 203.0979,  12.9857,  49.4429],
        [342.8600,  33.2996,  44.3301,  24.3918],
        [585.7931, 145.9501, 106.7544,  80.9010]], device='cuda:0')
tensor([[0.5578, 0.3837, 0.0655, 0.0902],
        [0.5619, 0.3451, 0.0431, 0.1153],
        [0.5845, 0.3173, 0.0203, 0.0773],
        [0.5357, 0.0520, 0.0693, 0.0381],
        [0.9153, 0.2280, 0.1668, 0.1264]], device='cuda:0')
tensor([[336.0611, 216.7296, 377.9647, 274.4376],
        [345.7985, 183.9637, 373.4114, 257.7541],
        [367.5749, 178.3765, 380.5606, 227.8194],
        [320.6949,  21.1037, 365.0250,  45.4956],
        [532.4159, 105.4997, 639.1703, 186.4006]], device='cuda:0')
tensor([[0.5251, 0.3386, 0.5906, 0.4288],
      